In [1]:
# imports
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# CA/NV
def ca_nv(dataarray):
    lat_drop = dataarray.where(dataarray.lat < 42, drop = True)
    lat_drop = lat_drop.where(lat_drop.lat > 31, drop = True)
    lon_drop = lat_drop.where(lat_drop.lon < -114, drop = True)
    return lon_drop

In [3]:
# reading in bias files
bias_next = ca_nv(xr.open_dataarray('./bias_17_noshft.nc'))
bias_prev = ca_nv(xr.open_dataarray('./bias_17_prev.nc'))

In [4]:
# reading in wrf files
next_06 = ca_nv(xr.open_dataarray('./wrf_06-Copy1.nc'))
next_07 = ca_nv(xr.open_dataarray('./wrf_07-Copy1.nc'))
next_08 = ca_nv(xr.open_dataarray('./wrf_08-Copy1.nc'))
next_09 = ca_nv(xr.open_dataarray('./wrf_09-Copy1.nc'))
next_10 = ca_nv(xr.open_dataarray('./wrf_10-Copy1.nc'))
next_11 = ca_nv(xr.open_dataarray('./wrf_11-Copy1.nc'))
next_12 = ca_nv(xr.open_dataarray('./wrf_12-Copy1.nc'))
next_13 = ca_nv(xr.open_dataarray('./wrf_13-Copy1.nc'))
next_14 = ca_nv(xr.open_dataarray('./wrf_14-Copy1.nc'))
next_15 = ca_nv(xr.open_dataarray('./wrf_15-Copy1.nc'))

prev_06 = ca_nv(xr.open_dataarray('./wrf_06_prev.nc'))
prev_07 = ca_nv(xr.open_dataarray('./wrf_07_prev.nc'))
prev_08 = ca_nv(xr.open_dataarray('./wrf_08_prev.nc'))
prev_09 = ca_nv(xr.open_dataarray('./wrf_09_prev.nc'))
prev_10 = ca_nv(xr.open_dataarray('./wrf_10_prev.nc'))
prev_11 = ca_nv(xr.open_dataarray('./wrf_11_prev.nc'))
prev_12 = ca_nv(xr.open_dataarray('./wrf_12_prev.nc'))
prev_13 = ca_nv(xr.open_dataarray('./wrf_13_prev.nc'))
prev_14 = ca_nv(xr.open_dataarray('./wrf_14_prev.nc'))
prev_15 = ca_nv(xr.open_dataarray('./wrf_15_prev.nc'))

In [5]:
np.count_nonzero(~np.isnan(bias_next.data.flatten())), np.count_nonzero(next_06.data.flatten())

(18863, 29920)

In [6]:
b_next = bias_next.data.flatten()
b_prev = bias_prev.data.flatten()
d6 = next_06.data.flatten()
d7 = next_07.data.flatten()
d8 = next_08.data.flatten()
d9 = next_09.data.flatten()
d10 = next_10.data.flatten()
d11 = next_11.data.flatten()
d12 = next_12.data.flatten()
d13 = next_13.data.flatten()
d14 = next_14.data.flatten()
d15 = next_15.data.flatten()
p6 = prev_06.data.flatten()
p7 = prev_07.data.flatten()
p8 = prev_08.data.flatten()
p9 = prev_09.data.flatten()
p10 = prev_10.data.flatten()
p11 = prev_11.data.flatten()
p12 = prev_12.data.flatten()
p13 = prev_13.data.flatten()
p14 = prev_14.data.flatten()
p15 = prev_15.data.flatten()

bn_data = {'Bias Next': b_next, 'Bias Previous': b_prev,
          'n6': d6, 'n7': d7, 'n8': d7, 'n9': d9, 'n10': d10, 'n11': d11, 'n12': d12, 'n13': d13, 'n14': d14, 'n15': d15,
          'p6': p6, 'p7': p7, 'p8': p7, 'p9': p9, 'p10': p10, 'p11': p11, 'p12': p12, 'p13': p13, 'p14': p14, 'p15': p15}
       
bn = pd.DataFrame(bn_data)
print(bn)

       Bias Next  Bias Previous          n6          n7          n8  \
0            NaN            NaN  290.151180  290.256460  290.256460   
1            NaN            NaN  290.176000  290.263178  290.263178   
2            NaN            NaN  290.182144  290.263604  290.263604   
3            NaN            NaN  290.172967  290.244670  290.244670   
4            NaN            NaN  290.188367  290.235032  290.235032   
...          ...            ...         ...         ...         ...   
29915  -1.534297      -1.671942  277.207247  275.411524  275.411524   
29916  -0.959545      -1.125353  278.537125  275.537709  275.537709   
29917  -0.969013      -1.121615  278.736580  275.533077  275.533077   
29918  -0.787131      -0.893907  279.816892  276.216736  276.216736   
29919   0.359095       0.308948  280.982735  277.899988  277.899988   

               n9         n10         n11         n12         n13  ...  \
0      289.963215  289.622879  288.962836  287.121402  287.502768  ...   

In [7]:
out_bias = pd.DataFrame()

def smallest_magnitude(next_dif, prev_dif, w_next, w_prev):
    if abs(prev_dif) < abs(next_dif):
        return w_prev
    else:
        return w_next
    
def output_bias(num_str):
    result = []
    for x in np.arange(29920):
        temp = smallest_magnitude(bn['Bias Next'][x], bn['Bias Previous'][x], bn['n'+num_str][x], bn['p'+num_str][x])
        result.append(temp)
    out_bias[num_str] = result

days = ['6','7','8','9','10','11','12','13','14','15']
for i in np.arange(10):
    output_bias(days[i])
print(out_bias)

                6           7           8           9          10          11  \
0      290.151180  290.256460  290.256460  289.963215  289.622879  288.962836   
1      290.176000  290.263178  290.263178  290.015999  289.619436  288.959444   
2      290.182144  290.263604  290.263604  290.009394  289.608809  289.018574   
3      290.172967  290.244670  290.244670  290.039924  289.584868  289.082376   
4      290.188367  290.235032  290.235032  290.029627  289.611887  289.186598   
...           ...         ...         ...         ...         ...         ...   
29915  277.207247  275.411524  275.411524  280.741594  277.725157  272.507570   
29916  278.537125  275.537709  275.537709  280.600389  279.325929  273.361078   
29917  278.736580  275.533077  275.533077  281.428500  280.362698  273.857685   
29918  279.816892  276.216736  276.216736  283.117589  281.761592  274.957530   
29919  282.089166  280.982735  280.982735  280.621608  284.474734  282.619252   

               12          

In [8]:
def diff(num_str, day, table, out):
    fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/ptt_2017.nc'
    liv_file = xr.open_dataset(fn)
    liv = ca_nv(liv_file['Tmax'][day]).data.flatten()
    table[num_str][table[num_str] == 0.0] = np.NAN
    table[num_str] = table[num_str] - 273.15
    subtr = np.subtract(np.array(table[num_str]), liv)
    out[num_str] = subtr

liv_days = [36,37,38,39,40,41,42,43,44,45][::-1] #experiment
for i in np.arange(10):
    diff(days[i], liv_days[i], out_bias, out_bias)
print(out_bias)

              6         7         8         9        10        11        12  \
0           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
29915 -3.847924 -1.593648 -0.093648  7.146423  1.469986 -9.087602 -6.650396   
29916 -2.984835 -2.314251 -0.494251  6.298429  2.373969 -8.590882 -4.470075   
29917 -3.087730 -2.941232 -0.841232  6.614191  2.938389 -8.306623 -4.986854   
29918 -2.800133 -3.320289 -0.980289  7.330564  3.384568 -7.939494 -5.342937   
29919  0.499201  2.662770  4.982770  4.501643  5.784770 -0.320713 -6.212706   

             13        14        15  
0           N

In [9]:
results = pd.DataFrame()

# average bias
sumz = 0
leng = 0
for i in days:
    temp = [x for x in np.array(out_bias[i]).flatten() if str(x) != 'nan']
    sumz += sum(temp)
    leng = len(temp)
avg_bias_best = sumz/(10*leng)
results['Best Bias: Bias'] = [avg_bias_best]

# rmse
def overall_rmse(dif_arr):
    sq_arr = []
    for i in dif_arr:
        sq_dif = np.square(i.data)
        sq_arr.append(sq_dif)
    arr_sq = [x for x in np.array(sq_arr).flatten() if str(x) != 'nan']
    div = sum(arr_sq) / (10*len(arr_sq))
    return np.sqrt(div)
dif_best = np.array([out_bias['6'], out_bias['7'], out_bias['8'],
         out_bias['9'], out_bias['10'],
         out_bias['11'], out_bias['12'],
         out_bias['13'], out_bias['14'],
         out_bias['15']]).flatten()
res = overall_rmse(dif_best)
results['Best Bias: RMSE'] = [res]

In [10]:
# prev
prev_out = pd.DataFrame()

prev_d = ['p6','p7','p8','p9','p10','p11','p12','p13','p14','p15']
for i in np.arange(10):
    diff(prev_d[i], liv_days[i], bn, prev_out)

summ = 0
for k in days:
    temp = [x for x in np.array(prev_out['p'+k]).flatten() if str(x) != 'nan']
    summ += sum(temp)
avg_bias_prev = summ/(10*leng)
results['Prev: Bias'] = [avg_bias_prev]

dif_prev = np.array([prev_out['p6'], prev_out['p7'], prev_out['p8'], prev_out['p9'],
                    prev_out['p10'], prev_out['p11'], prev_out['p12'], prev_out['p13'],
                    prev_out['p14'], prev_out['p15']])
res2 = overall_rmse(dif_prev)
results['Prev: RMSE'] = [res2]

In [11]:
# next
next_out = pd.DataFrame()

next_d = ['n6','n7','n8','n9','n10','n11','n12','n13','n14','n15']
for i in np.arange(10):
    diff(next_d[i], liv_days[i], bn, next_out)

summn = 0
for k in days:
    temp = [x for x in np.array(next_out['n'+k]).flatten() if str(x) != 'nan']
    summn += sum(temp)
avg_bias_next = summn/(10*leng)
results['Next: Bias'] = [avg_bias_next]

dif_next = np.array([next_out['n6'], next_out['n7'], next_out['n8'], next_out['n9'],
                    next_out['n10'], next_out['n11'], next_out['n12'], next_out['n13'],
                    next_out['n14'], next_out['n15']])
res3 = overall_rmse(dif_next)
results['Next: RMSE'] = [res3]
print(results)

   Best Bias: Bias  Best Bias: RMSE  Prev: Bias  Prev: RMSE  Next: Bias  \
0        -1.561094          1.53697   -1.788286    1.546433   -1.525651   

   Next: RMSE  
0    1.546566  


In [12]:
out_bias

,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
29915,-3.847924,-1.593648,-0.093648,7.146423,1.469986,-9.087602,-6.650396,-3.669784,0.133956,0.506665
29916,-2.984835,-2.314251,-0.494251,6.298429,2.373969,-8.590882,-4.470075,-2.122501,1.079054,1.059631
29917,-3.087730,-2.941232,-0.841232,6.614191,2.938389,-8.306623,-4.986854,-2.448049,0.730092,1.494515
29918,-2.800133,-3.320289,-0.980289,7.330564,3.384568,-7.939494,-5.342937,-3.129855,0.774932,1.580883


In [13]:
bias = out_bias.values.flatten()[~np.isnan(out_bias.values.flatten())].mean()
bias

-1.5610936533789976

In [14]:
sq_all = out_bias**2
sq_all

,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
29915,14.806522,2.539713,0.008770,51.071366,2.160858,82.584503,44.227771,13.467312,0.017944,0.256710
29916,8.909240,5.355759,0.244284,39.670210,5.635729,73.803259,19.981574,4.505010,1.164357,1.122818
29917,9.534075,8.650848,0.707672,43.747520,8.634132,68.999993,24.868711,5.992942,0.533035,2.233574
29918,7.840745,11.024317,0.960967,53.737174,11.455298,63.035566,28.546978,9.795991,0.600519,2.499190


In [15]:
rmse = np.sqrt(sq_all.values.flatten()[~np.isnan(sq_all.values.flatten())].mean())
rmse

4.860326207371779